## 14. 순환신경망 ##
### 14.1 순환뉴런 ###

![RNN1](images/rnn1.png)

<center><b style="font-size : 1.2em">순환 뉴런 & 타임 스텝으로 펼친 뉴런</b></center>

![RNN2](images/rnn2.png)

<center><b style="font-size : 1.2em">순환 뉴런의 층 & 타입 스텝으로 펼쳐진 층</b></center>

각 순환 뉴런은 두 개의 가중치 $\mathbf{w}_{x}$ 와 $\mathbf{w}_{y}$ 를 가지는데, $\mathbf{w}_{x}$ 는 $\mathbf{x}_{t}$ 를 위한 것이고 $\mathbf{w}_{y}$는 이전 타임 스텝의 출력 $\mathbf{y}_{t-1}$을 위한 것이다. 이것을 순환 층(layer) 전체로 생각하면 가중치 벡터 $\mathbf{w}_{x}$ 와 $\mathbf{w}_{y}$ 를 행렬 $\mathbf{W}_{x}$ 와 $\mathbf{W}_{y}$ 로 나타낼 수 있으며 다음의 식과 같이 표현할 수 있다.

$$
\mathbf{y}_{t} = \phi \left( \mathbf{W}_{x}^{T}\cdot\mathbf{x}_{t} + \mathbf{W}_{y}^{T} \cdot\mathbf{y}_{t-1} + \mathbf{b} \right)
$$

<br>

그리고 타임 스텝 $t$ 에서의 미니배치(mini-batch)의 입력을 행렬 $\mathbf{X}_{t}$ 로 나타내어 아래와 같이 순환 층의 출력을 한번에 계산할 수 있다.

$$
\mathbf{Y}_{t}= \phi \left( \mathbf{X}_{t} \cdot \mathbf{W}_{x} + \mathbf{Y}_{t-1} \cdot \mathbf{W}_{y} + \mathbf{b} \right)
$$

$$
= \phi \left( \begin{bmatrix} \mathbf{X}_{t} \mathbf{Y}_{t-1} \end{bmatrix}\begin{bmatrix} \mathbf{W}_{x} \\ \mathbf{W}_{y} \end{bmatrix} + \mathbf{b} \right)
$$

$$
여기서 W = \begin{bmatrix} \mathbf{W}_{x} \\ \mathbf{W}_{y} \end{bmatrix}
$$



$\phi$는 활성화 함수이다, RNN에서의 활성화 함수로는 $\tanh$ 가 주로 사용된다


<br/>

<ul>
<li>$\mathbf{Y}_{t}$ : 타임 스텝 $t$에서 미니배치에 있는 각 샘플(미니배치)에 대한 순환 층의 출력이며, <b style="color:red; font-size:1.1em">$m \times n_{\text{neurons}}$</b> 행렬($m$은 미니배치, $n_{\text{neurons}}$은 뉴런 수), <b style="background-color:#00C500">각 뉴런별로 출력값이 나오기때문에 샘플의 수 X 뉴런의 수</b></li>
<li>$\mathbf{X}_{t}$ : 모든 샘플의 입력값을 담고 있는 <b style="color:red; font-size:1.1em">$m \times n_{\text{inputs}}$</b> 행렬 ($n_{\text{inputs}}$은 입력 특성 수)</li>
<li>$\mathbf{W}_{x}$ : 현재 타임 스텝 $t$의 입력에 대한 가중치를 담고 있는 <b style="color:red; font-size:1.1em">$n_{\text{inputs}} \times n_{\text{neurons}}$</b> 행렬</li>
<li>$\mathbf{W}_{y}$ : 이전 타임 스텝 $t-1$ 의 출력에 대한 가중치를 담고 있는 <b style="color:red; font-size:1.1em">$n_{\text{neurons}} \times n_{\text{neurons}}$</b> 행렬</li>
<li>$\mathbf{b}$ : 각 뉴런의 편향(bias)을 담고 있는 $n_{\text{neurons}}$ 크기의 벡터</li>
<li>$\begin{bmatrix} \mathbf{X}_{t} \mathbf{Y}_{t-1} \end{bmatrix}$는 $\mathbf{X}_{t}\mathbf{Y}_{t-1}$을 수평적으로 연결한다는 것을 의미</li>
</ul>
위의 식에서 $\mathbf{Y}_{t}$ 는 $\mathbf{X}_t$ 와 $\mathbf{Y}_{t-1}$의 함수이므로, 타임 스텝 $t=0$ 에서부터 모든 입력에 대한 함수가 된다. 첫 번째 타임 스텝인 $t=0$ 에서는 이전의 출력이 없기 때문에 일반적으로 $0$으로 초기화 한다.

### 입력과 출력 시퀀스 ###

![RNN4](images/rnn4.png)


<ul>
    <li><p><b>Vector to Sequence</b> : 첫 번째 타임 스텝에서 단 하나의 입력만 네트워크에 주입하고(다른 모든 타임 스텝에서는 0 주입) 출력시퀀스 연결, 예)이미지 캡션 출력</p></li>
    <li><p><b>Sequence to Vector</b> : 입력시퀀스를 네트워크에 주입하고, 마지막을 제외한 모든 출력을 무시, 예)감성점수 출력</p></li>
    <li><p><b>Sequence to Sequence</b> : 입력시퀀스를 받아 출력시퀀스를 만듬, 예)주식가격 예측</p></li>
    <li><p><b>Delayed Sequence to Sequence</b> : 시퀀스-투-벡터(인코더) 뒤에 벡터-투-시퀀스(디코더) 연결, 예)문장 번역</p></li>
</ul>

### 14.2 텐서플로로 기본 RNN 구성하기 ###

In [ ]:
n_inputs = 3 #입력 특성의 개수
n_neurons = 5 #순환 뉴런층 안에 있는 뉴런의 개수

X0 = tf.placeholder(tf.float32, [None, n_inputs])
#... 그래픽카드 오면 계속하기...